In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from scipy import ndimage
from matplotlib import pyplot
import scipy
from PIL import Image
from skimage import measure
import math
from tqdm import tqdm_notebook as tqdm
from skimage import morphology,draw
import os
import pandas as pd
%pylab inline
from imageio import imread
import pickle

/home/xupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['draw']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# load a single image to determine image size
f = Image.open("mask41noassign.vsseg_export_s100.png")
im = np.asarray(f)
im.shape

In [ ]:
# z is section number
# n is the original segmentation image volume
z = 31
n = np.zeros((z,im.shape[0],im.shape[1]),dtype = 'uint8')

In [ ]:
for i in range(z):
    f = Image.open("mask41noassign.vsseg_export_s" +str("%03d"%j)+".png")
    im = np.asarray(f)
    
    # if empty section, copy from the section above
    if np.sum(im) == 0:
        im = n[i-1]
        
    n[i] = im

# main function

In [310]:
get_point_annotation(miplevel=3,export_x=6881,export_y=2525,muscleind=3,axonind='1a')

array([  65,  772, 1007])

In [307]:
imgs_.shape

(300, 1681, 2936)

In [311]:
imgs_[65,  772, 1007]

21

In [315]:
diameter(imgs_,3,21,np.array([65, 772,1007]),
         1,0.1)

begin 21
initiate 21
after 1
1



2.03125 12.1875


14.21875

In [ ]:
def diameter(segmentationVolume,mipLevel,axonId,point,degreeAccuracy,distanceAccuracy):
    '''
    calculate the diameter of an axon at a given point
    
    input:
        segmentationVolume: [3D array] the original segmentation image volume
        axonIdd: [int] the Id of desired axon.
        point: [1*3 array] the point (z,x,y) passing which we want a cross section and its diameter. 
        mipLevel: [int] the mipLevel of original segmentation image.
        degreeAcuracy: [int] degree accuracy of the diameter.
        distanceAccuracy: [int] accuracy of the length of the diameter.
    output:
        diameter1: [int] the diameter on the first axis of this axon at this point
        diameter2: [int] the diameter on the second axis of this axon at this point
    '''
        
    axon = preprocessing(segmentationVolume,axonId,mipLevel)
    
    #find the minimum distance
    vector = minDistance(axon,point,degreeAccuracy,distanceAccuracy)
    
    #redefine the centerPoint, this time closer to the center
    minDis = calDiameter(axon,point,vector,distanceAccuracy)[0]
    reverseMinDis = calDiameter(axon,point,vector,distanceAccuracy)[1]
    centerPoint = point + ((minDis-reverseMinDis)/2)*vector
    
    #find the first axis defined by the minimum distance passing the centerPoint
    firstAxis = minDistance(axon,centerPoint,degreeAccuracy,distanceAccuracy)
    diameter1 = calDiameter(axon,centerPoint,firstAxis,distanceAccuracy)[2]
    
    #find the second axis, passing the centerPoint, orthogonal to the first axis 
    secondAxis = findSecondAxis(axon,centerPoint,degreeAccuracy,distanceAccuracy,firstAxis)
    diameter2 = calDiameter(axon,centerPoint,secondAxis,distanceAccuracy)[2]

    return diameter1,diameter2,(diameter1+diameter2)/2

below are all functions called by the main function


# data pre-processing (preserve the desired axon and interpolate)

In [83]:
def preprocessing(segmentationVolume,axonId,mipLevel,point):
    '''
    preserve the desired axon and interpolate
    
    input: 
        segmentationVolume: [3D array] the original segmentation image volume.
        axonId: [int] the Id of desired axon.
        mipLevel: [int] the mipLevel of original segmentation image.
    output:
        interpolatedAxon: [3D array] only containing the disired axon, and interpolated to mip0 resolution.
    '''
    
    print('initiate',segmentationVolume[point[0],point[1],point[2]])
    #preserve the desired axon
    zMIN = max(0,(point[0]-int(200/int((16/(2**mipLevel))))))
    zMAX = min(segmentationVolume.shape[0],(point[0]+int(200/int((16/(2**mipLevel))))+1))
    xMIN = max(0,(point[1]-200))
    xMAX = min(segmentationVolume.shape[1],(point[1]+201))
    yMIN = max(0,(point[2]-200))
    yMAX = min(segmentationVolume.shape[2],(point[2]+201))
    axonVolume = (segmentationVolume[zMIN:zMAX,xMIN:xMAX,yMIN:yMAX] == axonId).astype('uint8')
    #print (axonVolume)
    #initiate the interpolated volume
    z = axonVolume.shape[0] * int((16/(2**mipLevel)))
    x = axonVolume.shape[1]
    y = axonVolume.shape[2]
    interpolatedAxon = np.zeros((z,x,y),dtype = 'uint8')
    
    #interpolation
    for i in range(y):
        res = cv2.resize(axonVolume[:,:,i],(x,z),interpolation = cv2.INTER_LINEAR)
        interpolatedAxon[:,:,i]=res
    
    point[0]=(point[0]-zMIN)*int((16/(2**mipLevel)))
    point[1]=point[1]-xMIN
    point[2]=point[2]-yMIN
    print('after',interpolatedAxon[point[0],point[1],point[2]])
    return interpolatedAxon,point

# diameter measurement

### get the direction with minimum distance between the axon boundry at this direction and the preset point

In [84]:
def minDistance(imgVolume,point,degreeAccuracy,distanceAccuracy):
    '''
    get the direction with minimum distance between the axon boundry at this direction and the preset point
    
    input:
        imgVolume: [3D array] interpolatedAxon.
        point: [1*3 array] the point passing which we want a cross section and its diameter. 
        degreeAcuracy: [int] degree accuracy of the direction with minimum distance.
        distanceAccuracy: [int] accuracy when calculating the distance between the axon boundry and the point.
    output:
        vector: [1*3 array] the direction with minimum diatance, representing by a unit vector.
    '''
    
    vector = np.array([0,0,0])
    min_dis = 99999
    counts = int(360/degreeAccuracy)
    for i in tqdm(range(counts)):
        for j in range(counts):
            a = (i-counts/2)/(counts/2)*math.pi
            b = (j-counts/2)/(counts/2)*math.pi
            z = math.sin(a)
            x = math.cos(a)*math.cos(b)
            y = math.cos(a)*math.sin(b)
            vec = np.array([z,x,y])
            dis = distance(imgVolume,point,vec,distanceAccuracy)
            if dis < min_dis:
                min_dis = dis
                vector = vec
    return vector

### get the second axis orthogonal to the first axis

In [ ]:
def findSecondAxis(imgVolume,point,degreeAccuracy,distanceAccuracy,firstAxis):
    '''
    get the sencond axis, which passes the point, orthogonal to the first axis, and have the minimum distance among 
    all orthogonal directions.
    
    input:
        imgVolume: [3D array] interpolatedAxon.
        point: [1*3 array] the point passing which we want a cross section and its diameter. 
        degreeAcuracy: [int] degree accuracy of the direction with minimum distance.
        distanceAccuracy: [int] accuracy when calculating the distance between the axon boundry and the point.
        firstAxis:[1*3 array] the vector which the secondAxis should be orthogonal to.
    output:
        secondAxis: [1*3 array] the direction of the secondAxis, with minimum diatance among all direction orthogonal 
                            to the firstAxis, representing by a unit vector.                  
    '''

    vector = np.array([0,0,0])
    min_dis = 99999
    counts = int(360/degreeAccuracy)
    for i in tqdm(range(counts)):
        for j in range(counts):
            a = (i-counts/2)/(counts/2)*math.pi
            b = (j-counts/2)/(counts/2)*math.pi
            z = math.sin(a)
            x = math.cos(a)*math.cos(b)
            y = math.cos(a)*math.sin(b)
            vec = np.array([z,x,y])
            if np.abs(vec.dot(firstAxis)) > 0.0088 :
                continue
            dis = distance(imgVolume,point,vec,distanceAccuracy)
            if dis < min_dis:
                min_dis = dis
                vector = vec
    return vector

### calculate the distance between the axon boundry at a preset direction and  a preset point

In [85]:
def distance(imgVolume,point,vector,accuracy):
    '''
    calculate the distance between the axon boundry at a preset direction and  a preset point
    
    input:
        imgVolume: [3D array] interpolatedAxon.
        point: [1*3 array] the point passing which we want a cross section and its diameter. 
        vector: [1*3 array] the preset direction
        accuracy: [int] accuracy when calculating the distance between the axon boundry and the point.
    output:
        distance: [int] the distance between the axon boundry at the vector direction and the point.
    '''
    
    dis = 0
    
    #firstly, define a range of the distance
    begin = 0
    end = 990
    for i in range(100):
        d = i*10.0
        newpoint = point + d*vector
        try:
            if imgVolume[math.floor(newpoint[0]),math.floor(newpoint[1]),math.floor(newpoint[2])] == 1:
                begin = d
                end = d
            else:
                end = d
                break
        except:
            break
            
    #then, use binary search to define the accurate distance with the preset accuracy
    for i in range(math.floor(math.log(10/accuracy,2)+1)):
        dmed2 = (begin + end)/2.0
        dmed1 = dmed2-accuracy
        med2 = point + dmed2 * vector
        med1 = point + dmed1 * vector
        if imgVolume[math.floor(med2[0]),math.floor(med2[1]),math.floor(med2[2])] == 0:
            if imgVolume[math.floor(med1[0]),math.floor(med1[1]),math.floor(med1[2])] == 1:
                dis = dmed2
                break
            else:
                end = dmed2
        else:
            begin = dmed2
        #if not found, it means the axon boundry at this direction is not contained in this imagevolume
        if i == math.floor(math.log(10/accuracy,2)+1)-1:
            dis = dmed2
            
    return dis

### calculate the diameter at the cross section defined by a point and a vector.

In [86]:
def calDiameter(imgVolume,point,vector,accuracy):
    '''
    calculate the diameter at the cross section defined by a point and a vector.
    
    input: 
        imgVolume: [3D array] interpolatedAxon.
        point: [1*3 array] the point passing which we want a cross section and its diameter. 
        vector: [1*3 array] the direction with the minimum distance from the boundry to the point
        accuracy: [int] accuracy when calculating the distance between the axon boundry and the point.
    output:
        dis1: [int] the distance on the vector direction
        dis2: [int] the distance on the reverse extension direction
        dis1+dis2: [int] the diameter
    '''
        
    dis1 = distance(imgVolume,point,vector,accuracy)
    dis2 = distance(imgVolume,point,-1*vector,accuracy)
    return dis1,dis2,dis1+dis2

### bounding box

In [87]:
def boundingBox(imgVolume):
    zMIN = 0
    zMAX = imgVolume.shape[0]-1
    for i in range(imgVolume.shape[0]):
        sum = np.sum(imgVolume[i,:,:])
        if zMIN == 0 and sum > 0:
            zMIN = i
        if sum > 0:
            zMAX = i
            
    xMIN = 0
    xMAX = imgVolume.shape[1]-1
    for i in range(imgVolume.shape[1]):
        sum = np.sum(imgVolume[:,i,:])
        if xMIN == 0 and sum > 0:
            xMIN = i
        if sum > 0:
            xMAX = i
    
    yMIN = 0
    yMAX = imgVolume.shape[2]-1
    for i in range(imgVolume.shape[2]):
        sum = np.sum(imgVolume[:,:,i])
        if yMIN == 0 and sum > 0:
            yMIN = i
        if sum > 0:
            yMAX = i
            
    return zMIN,zMAX,xMIN,xMAX,yMIN,yMAX